<a href="https://colab.research.google.com/github/William-Schwarz/Decifra-Contrato/blob/william_ds/Decifra_Contrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip -q install google-genai google-adk PyPDF2 python-docx

In [72]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [73]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [74]:
from google.colab import files
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [75]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [76]:
def ler_arquivo(nome_arquivo):
    try:
        if nome_arquivo.endswith(".pdf"):
            with open(nome_arquivo, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    text += page.extract_text()
                return text
        elif nome_arquivo.endswith(".docx"):
            document = Document(nome_arquivo)
            text = ""
            for paragraph in document.paragraphs:
                text += paragraph.text + "\n"
            return text
        elif nome_arquivo.endswith(".txt"):
            with open(nome_arquivo, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return "Formato de arquivo não suportado."
    except Exception as e:
        return f"Erro ao ler o arquivo: {e}"

In [77]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [78]:
######################################################
# --- Agente 1: Receptor e Extrator de Contrato --- #
######################################################
def agente_receptor():
    receptor = Agent(
        name="agente_receptor",
        model=MODEL_ID,
        instruction="""
        Você é o agente inicial do sistema Decifra-Contrato. Sua tarefa é receber o contrato fornecido pelo usuário.
        Este contrato pode ser um texto diretamente inserido ou o conteúdo de um arquivo.

        Uma vez que o contrato é fornecido, sua principal função é extrair o texto completo e limpo do contrato.
        Remova qualquer formatação desnecessária que possa dificultar a análise posterior (como quebras de linha excessivas, numerações automáticas, etc.).

        Se o usuário fornecer informações adicionais sobre o tipo de contrato (por exemplo, "contrato de aluguel", "termo de serviço"), anote essa informação para auxiliar os próximos agentes.

        Ao final, entregue o texto puro do contrato para o próximo agente.
        """,
        description="Agente que recebe e extrai o texto do contrato."
    )
    return receptor

In [79]:
#######################################################
# --- Agente 2: Identificador de Jargões --- #
#######################################################
def agente_identificador():
    identificador = Agent(
        name="agente_identificador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em identificar jargões jurídicos e termos técnicos dentro de um texto de contrato.
        Sua tarefa é analisar o texto do contrato fornecido e listar todos os termos que provavelmente seriam desconhecidos por uma pessoa leiga.

        Considere termos como: latim jurídico (e.g., 'ad hoc'), termos técnicos específicos da área do contrato (e.g., em um contrato de tecnologia, 'API', 'SaaS'), e cláusulas com redação complexa.

        Apresente a lista de jargões identificados de forma clara e concisa para o próximo agente.
        """,
        description="Agente que identifica jargões jurídicos e técnicos."
    )
    return identificador

In [80]:
########################################################
# --- Agente 3: Explicador de Termos --- #
########################################################
def agente_explicador(google_search):
    explicador = Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em explicar termos jurídicos e técnicos de forma clara e acessível para pessoas leigas.
        Você receberá uma lista de jargões identificados em um contrato.

        Para cada termo da lista, sua tarefa é fornecer uma explicação concisa e em linguagem simples.
        Se necessário, utilize a ferramenta de busca do Google (google_search) para encontrar definições e explicações adequadas.

        Tente fornecer exemplos práticos ou analogias para facilitar a compreensão.

        Apresente cada termo seguido de sua explicação de forma clara e organizada.
        """,
        description="Agente que explica jargões jurídicos e técnicos para leigos.",
        tools=[google_search]
    )
    return explicador

In [81]:
#########################################################
# --- Agente 4: Resumidor e Destacador de Pontos Chave --- #
#########################################################
def agente_resumidor():
    resumidor = Agent(
        name="agente_resumidor",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em resumir contratos e destacar os pontos mais importantes para uma pessoa leiga.
        Com base no texto completo do contrato, sua tarefa é gerar um resumo conciso, com linguagem simples e direta.

        Além do resumo, identifique e liste as cláusulas ou pontos que são cruciais para o entendimento do usuário, como:
        - Objeto do contrato
        - Obrigações das partes
        - Prazos e condições
        - Formas de pagamento (se aplicável)
        - Condições de rescisão
        - Penalidades (se houver)

        Apresente o resumo e os pontos chave de forma clara e organizada, utilizando marcadores ou listas para facilitar a leitura.
        """,
        description="Agente que resume contratos e destaca pontos chave."
    )
    return resumidor

In [82]:
print("📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐")

# --- Obter o Contrato do Usuário ---
opcao = input("❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? ")
contrato_texto = ""

if opcao.lower() == 'texto':
    contrato_texto = input("Cole o texto do contrato aqui: ")
elif opcao.lower() == 'arquivo':
    uploaded = files.upload()
    if uploaded:
        nome_arquivo = list(uploaded.keys())[0]
        contrato_texto = ler_arquivo(nome_arquivo)
        print(f"\nArquivo '{nome_arquivo}' lido com sucesso.")
    else:
        print("\nNenhum arquivo foi carregado.")
else:
    print("\nOpção inválida. Por favor, digite 'texto' ou 'arquivo'.")

if contrato_texto:
    print("\n✨ Processando o contrato. Aguarde...\n")

    # Inicializa os agentes
    receptor = agente_receptor()
    identificador = agente_identificador()
    explicador = agente_explicador(google_search)
    resumidor = agente_resumidor()

    # --- Execução dos Agentes ---

    # Agente Receptor
    mensagem_receptor = contrato_texto if opcao.lower() == 'texto' else f"Arquivo carregado: {contrato_texto}"
    texto_contrato_extraido = call_agent(receptor, mensagem_receptor)
    print("--- 📝 Contrato Extraído ---")
    display(to_markdown(texto_contrato_extraido))
    print("-----------------------------\n")

    # Agente Identificador de Jargões
    jargoes_identificados = call_agent(identificador, texto_contrato_extraido)
    print("--- 🔎 Jargões Identificados ---")
    display(to_markdown(jargoes_identificados))
    print("-------------------------------\n")

    # Agente Explicador de Termos
    explicacoes_termos = call_agent(explicador, jargoes_identificados)
    print("--- 💡 Explicações dos Termos ---")
    display(to_markdown(explicacoes_termos))
    print("---------------------------------\n")

    # Agente Resumidor e Destacador de Pontos Chave
    resumo_contrato = call_agent(resumidor, texto_contrato_extraido)
    print("--- 🔑 Resumo e Pontos Chave do Contrato ---")
    display(to_markdown(resumo_contrato))
    print("-------------------------------------------\n")

    print("✅ Análise do Contrato Concluída!")
else:
    print("Processamento cancelado devido à falta de contrato.")

📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐
❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? arquivo


Saving MINUTA_DE_CONTRATO_-_PREGO_49_-_MATERIAL_DE_INFORMTICA.pdf to MINUTA_DE_CONTRATO_-_PREGO_49_-_MATERIAL_DE_INFORMTICA (2).pdf

Arquivo 'MINUTA_DE_CONTRATO_-_PREGO_49_-_MATERIAL_DE_INFORMTICA (2).pdf' lido com sucesso.

✨ Processando o contrato. Aguarde...

--- 📝 Contrato Extraído ---


> ```text
> CONTRATO QUE ENTRE SI CELEBRAM O CONSELHO NACIONAL DO MINISTÉRIO PÚBLICO – CNMP E A PESSOA JURÍDICA ___________________ PARA AQUISIÇÃO DE MATERIAL DE INFORMÁTICA NA FORMA ABAIXO:
> 
> O CONSELHO NACIONAL DO MINISTÉRIO PÚBLICO , CNPJ n.º 11.439.520/0001-11, situado no SHIS, QI 03, lote A, blocos B, E e G, - Lago Sul - Brasília/DF, representado neste ato por sua Secretária de Administração, Sra. SÔNIA MÁRCIA FERNANDES AMARAL , brasileira, divorciada, servidora pública, RG 980.300 – SSP/DF, CPF: 386.128.521-53 no uso da competência que lhe foi atribuída pela Portaria CNMP n .º 94, de 14 de dezembro de 2010, ou, nas ausências e impedimentos desta, pelo seu substituto, Sr. EDIMILSON AVELINO DA SILVA, brasileiro, divorciado, servidor público, RG: 2.620.787 – IFP/RJ, CPF: 219.676.957-68, conforme Portaria CNMP nº 10, de 22 de fevereiro de 2011, ambos residentes e domiciliados nesta Capital, doravante denominado simplesmente CONTRATANTE e a pessoa jurídica ________________________________, CNPJ n.º ________ ___________, estabelecida _______________________________, CEP: ________, Cidade/Estado , neste ato representada por ________________________, solteiro/casado , profissão , inscrito no RG sob o nº __________________, e no CPF sob o nº _______________, residente e domiciliado na _____________________, CEP: ________, Cidade/Estado , e daqui por diante designada simplesmente CONTRATADA , tendo em vista o contido no Processo CNMP n.º 0.00.002.001424/2011-32, referente ao Pregão Eletrônico CNMP n.º 49/2011, considerando as disposições estabelecidas na Lei nº 8.666 de 21/06/1993, Lei nº 10.520 de 17/07/2002, e ainda, pelos Decreto nº 3.555/2000, Decreto nº 5.450/2005, Decreto nº 5.504/2005, Lei Complementar nº 123 de 14/12/2006, e demais normas pertinentes, têm entre si, justo e avençado, e celebram o presente Contrato mediante as seguintes cláusulas e condições:
> 
> CLÁUSULA PRIMEIRA – DO OBJETO
> 1.1 Constitui objeto do presente Contrato a aquisição de (descrever o objeto da contratação) , incluída a prestação de garantia e assistência técnica, em conformidade com as especificações e detalhamento do Edital que rege o Pregão Eletrônico CNMP nº 49/2011, os Termos de Referência e de Garantia e Assistência Técnica que o acompanham e a proposta da CONTRATADA, os quais, independentemente de transcrição, são partes integrantes deste instrumento e serão observados naquilo que não o contrarie.
> 
> CLÁUSULA SEGUNDA – DO REGIME DE EXECUÇÃO
> 2.1. A execução do objeto do presente Contrato será de forma indireta, por empreitada por preço unitário, em conformidade com o disposto na Lei n.º 8.666/1993.
> 
> CLÁUSULA TERCEIRA – DAS OBRIGAÇÕES DA CONTRATADA 3.1. Constituem obrigações da CONTRATADA:
> 3.1.1. Entregar os objetos do contrato conforme especificado e dentro do prazo de entrega estipulado neste instrumento;
> 3.1.2. Prestar garantia e assistência técnica conforme estabelecido no Termo de Referência e no Termo de Garantia Técnica;
> 3.1.3. Efetuar o pagamento de todos os impostos, taxas e demais obrigações fiscais incidentes ou que vierem a incidir sobre o objeto do contrato, até o recebimento definitivo do objeto;
> 3.1.4. Apresentar as Notas Fiscais/Faturas contendo a discriminação exata e os respectivos quantitativos, com os valores contratados;
> 3.1.5. Manter, durante toda a execução do Contrato, as condições de habilitação e qualificação exigidas para a contratação;
> 3.1.6. Manter sigilo, sob pena de responsabilidade, sobre todo e qualquer assunto de interesse do CONTRATANTE ou de terceiros, de que tomar conhecimento em razão da execução contratual, devendo orientar seus empregados nesse sentido;
> 3.1.7. Prestar as informações e os esclarecimentos que venham a ser solicitados pelos técnicos do CNMP, atendendo de imediato às reclamações;
> 3.1.8. Arcar com todas as despesas decorrentes dos produtos fornecidos;
> 3.1.9. Entregar o objeto do contrato de acordo com o especificado, dentro do prazo de entrega conforme Cláusula 6.2;
> 3.1.10. Entregar ao gestor do Contrato, no Conselho Nacional do Ministério Público, o Termo de Recebimento Provisório referente à entrega dos equipamentos;
> 3.1.11. Assinar e cumprir, durante o período de garantia, o estabelecido no Termo de Garantia e Assistência Técnica, o qual faz parte integrante deste instrumento.
> 3.2. O acompanhamento e fiscalização da execução do Contrato serão exercidos por servidores posteriormente designados pela CONTRATANTE, por meio de portaria específica.
> CLÁUSULA QUARTA - DA GARANTIA E DA ASSISTÊNCIA TÉCNICA
> 4.1. A garantia e assistência técnica serão prestadas conforme o Termo de Garantia e Assistência Técnica (Edital do Pregão Eletrônico nº 49/2011 - Anexo IV).
> 
> CLÁUSULA QUINTA – DAS OBRIGAÇÕES DA CONTRATANTE
> 5.1. Constituem obrigações do CONTRATANTE:
> 5.1.1 Permitir o acesso dos representantes ou profissionais da CONTRATADA ao local de entrega do objeto, desde que devidamente identificados;
> 5.1.2. Atestar a execução do objeto deste Contrato por meio do gestor/comissão;
> 5.1.3. Efetuar o pagamento à CONTRATADA de acordo com as condições estabelecidas neste Contrato;
> 5.1.4. Acompanhar e fiscalizar a execução do objeto , registrar as ocorrências e eventuais deficiências relacionadas com a execução, sob os aspectos quantitativo e qualitativo, e comunicar as ocorrências de quaisquer fatos que exijam medidas corretivas por parte da CONTRATADA;
> 5.1.5. Designar, por meio de portaria específica, servidor e seu substituto em caso de impedimentos e afastamentos legais, para proceder à fiscalização e acompanhamento da execução deste contrato.
> 5.1.6. Proporcionar todas as facilidades indispensáveis à boa execução dos serviços.
> 5.1.7 Relacionar-se com a CONTRATADA exclusivamente por meio de pessoa por ela indicada;
> 5.1.8 O CONTRATANTE, por meio da Coordenadoria de Administração, reserva-se ao direito de exercer, quando lhe convier, fiscalização sobre a execução deste Contrato e, ainda, aplicar multa ou rescindir o Contrato, caso a CONTRATADA desobedeça a qualquer das Cláusulas estabelecidas neste instrumento.
> 
> CLÁUSULA SEXTA – DO LOCAL E PRAZOS DE ENTREGA
> 6.1. O equipamento será entregue no Almoxarifado do Conselho Nacional do Ministério Público, localizado na Sede Administrativa, ou em outro local, em Brasília-DF, a ser indicado pelo CNMP e sem ônus para este.
> 6.2. O prazo de entrega será de 30 (trinta dias) dias corridos e contados da confirmação do recebimento do pedido de fornecimento.
> 6.3. O bem deverá ser entregue, instalado, em perfeito estado de funcionamento e sem marcas, amassados, arranhões ou outros problemas.
> 
> CLAÚSULA SÉTIMA – DO RECEBIMENTO
> 7.1. Os equipamentos serão aceitos, mediante elaboração de relatório, da seguinte forma:
> 7.1.1. O recebimento dos bens se dará, provisoriamente, em até 5 (cinco) dias úteis, a partir da entrega no almoxarifado do CNMP, na Sede Administrativa, para posterior verificação da conformidade com as especificações contidas neste contrato e no Termo de Referência;
> 7.1.2. O recebimento definitivo dar-se-á no prazo de 5 (oito) dias úteis após o recebimento provisório, para verificação da quantidade e qualidade do material e consequente aceitação.
> 7.2 Após entrega dos bens e antes do recebimento definitivo, se houver problema com os equipamentos entregues, eles deverão ser substituídos por outros equipamentos novos e sem uso, sem ônus para o CNMP.
> CLÁUSULA OITAVA - DO PRAZO DE VIGÊNCIA
> 8.1 - O presente Contrato terá vigência da data da sua assinatura até a emissão do Termo de Recebimento Definitivo pelo CNMP, nos prazos e condições estabelecidos no Termo de Referência que faz parte integrante deste instrumento, ressalvado o período de garantia e assistência técnica, conforme estabelecido no itens 3.1.11 e 4.1 deste Contrato e o disposto no Termo de Garantia e Assistência Técnica.
> 
> CLÁUSULA NONA – DO VALOR 9.1. O valor total do presente Contrato é de R$ XXX ,00 ( _________ reais), conforme discriminado na proposta comercial da contratada, vencedora da Licitação (Edital nº 49/2011), que passa a integrar o presente Contrato, independentemente de transcrição.
> 9.2 Já estão incluídas no preço total todas as despesas de frete, embalagens, impostos, transporte, mão de obra e demais encargos indispensáveis ao perfeito cumprimento das obrigações decorrentes deste Contrato, assim como as despesas relativas à assistência técnica e garantia.
> 
> CLÁUSULA DÉCIMA - DO PAGAMENTO
> 10.1 O pagamento será efetuado a favor da CONTRATADA em até 10 (dez) dias úteis, após recebimento definitivo do material, mediante apresentação da respectiva nota fiscal devidamente atestada pelo setor competente, por meio de Ordem Bancária e de acordo com as condições constantes na proposta da empresa e aceito pelo CNMP.
> 10.2 Na nota fiscal deverá constar o Endereço, CNPJ e o domicílio bancário da CONTRATADA, bem como o número da Nota de Empenho e a descrição clara do objeto, com valores expressos em moeda corrente nacional.
> 10.3 A CONTRATADA deverá fazer constar como beneficiário/cliente da Nota Fiscal/Fatura correspondente, emitida sem rasuras, o Conselho Nacional do Ministério Público, CNPJ nº 11.439.520/0001-11.
> 10.4 Sobre o valor da Nota Fiscal, a CONTRATANTE fará as retenções devidas ao INSS e as dos impostos e contribuições previstas na Instrução Normativa SRF nº 480, de 15.12.2004 e 539, de 25.04.2005 e Lei Complementar nº 116/2003.
> 10.5 Caso a CONTRATADA seja optante pelo Sistema Integrado de Pagamento de Impostos e Contribuições da Microempresas e Empresas de Pequeno Porte – “SIMPLES” (Lei nº 9.317/96), não serão feitas as retenções de que tratam as citadas instruções normativas, ficando a CONTRATADA nesse caso obrigada a apresentar declaração, na forma do Anexo IV da Instrução Normativa SRF nº 480, de 15/12/2004, em duas vias, assinadas pelo seu representante legal.
> 10.6 A consulta relativa à regularidade fiscal, exigida quando da habilitação, será feita previamente ao pagamento, devendo seu resultado ser juntado aos autos do processo próprio e ficando o efetivo pagamento a ela condicionado.
> 10.7 Nenhum pagamento será efetuado à CONTRATADA, enquanto pendente de liquidação qualquer obrigação financeira que lhe for imposta, em virtude de penalidade ou inadimplência contratual, sem que isso gere direito a acréscimos de qualquer natureza.
> 10.8 Ao CNMP fica reservado o direito de não efetuar o pagamento se, no momento da aceitação, os bens fornecidos, não estiverem em perfeitas condições e em conformidade com as especificações estipuladas.
> 10.9 Nos casos de eventuais atrasos de pagamento, desde que a Contratada não tenha concorrido de alguma forma para tanto, fica convencionada a taxa de atualização financeira devida pelo Conselho Nacional do Ministério Público, conforme disposto no art. 36, § 4º, da Instrução Normativa/SLTI- MP n.º 02, de 30/04/2008, mediante a aplicação da seguinte fórmula: EM = I x N x VP, sendo
> I = (TX/100) , assim apurado: I = (6/100) I = 0,00016438
> 365 365
> Em que:
> I = Índice de atualização financeira ;
> TX = Percentual da taxa de juros de mora anual = 6%;
> EM = Encargos moratórios;
> N = Número de dias entre a data prevista para o pagamento e a do efetivo pagamento;
> VP = Valor da parcela em atraso.
> 10.10 Aplica-se a mesma regra disposta no parágrafo anterior, na hipótese de eventual pagamento antecipado, observado o disposto no art. 38 do Decreto nº 93.872/86.
> 
> CLÁUSULA DÉCIMA PRIMEIRA – DA DOTAÇÃO ORÇAMENTÁRIA
> 11.1 As despesas com a execução dos serviços, objeto deste termo, correrão, no presente exercício, à conta da Natureza de Despesa 3.3.90.30 , do Programa/Atividade 03.062.0581.8010.0001, constante do Orçamento Geral da União para este fim :
> 11.2 Para cobertura da despesa foi emitida a Nota de Empenho nº XXXXXX, de ___ de ______ de 2011.
> 
> CLÁUSULA DÉCIMA SEGUNDA – DA ALTERAÇÃO
> 12.1 Este Contrato poderá, nos termos do art. 65 da Lei nº 8.666/93, ser alterado por meio de Termos Aditivos, objetivando promover os acréscimos ou supressões que se fizerem necessários.
> 12.2 Nenhum acréscimo ou supressão poderá exceder o limite estabelecido no § 1º do art. 65 da Lei nº 8.666/93, salvo as supressões resultantes de acordos celebrados entre os contratantes .
> 
> CLÁUSULA DÉCIMA TERCEIRA – DAS PENALIDADES E RECURSOS
> 13.1. Conforme o disposto no art. 28 do Decreto nº 5.450, de 31/05/2005, aquele que, convocado dentro do prazo de validade de sua proposta, não assinar o contrato ou ata de registro de preços, deixar de entregar documentação exigida no edital, apresentar documentação falsa, ensejar o retardamento da execução de seu objeto, não mantiver a proposta, falhar ou fraudar na execução do contrato, comportar-se de modo inidôneo, fizer declaração falsa ou cometer fraude fiscal, garantido o direito à ampla defesa, ficará impedido de licitar e de contratar com a União, e será descredenciado no SICAF, pelo prazo de até cinco anos, sem prejuízo das multas previstas no edital e no contrato e das demais cominações legais;
> 13.2. Além do previsto no subitem anterior, pela inexecução total ou parcial do objeto do Contrato e pela verificação de quaisquer das situações previstas no art. 78, incisos I a XI da Lei nº 8.666/93, a Contratante poderá aplicar as seguintes penalidades, sem o prejuízo de outras: 13.2.1. Advertência;
> 13.2.2 Multa moratória, em caso de atraso na entrega dos equipamentos, será cobrada multa no valor de 0,7% por dia de atraso, calculada sobre o valor total dos equipamentos em mora, limitada a 30 (trinta) dias.
> 13.2.3 Multa, em caso de atraso na solução do atendimento técnico, será cobrada multa no valor de valor de 0,3% por dia de atraso para cada chamado não solucionado, calculada sobre o valor do equipamento, limitado a 30 (trinta) dias. Poderá haver mais de um chamado aberto simultaneamente por equipamento, razão pela qual poderá haver a cobrança cumulativa de multas sobre o atraso no atendimento técnico.
> 13.2.4 Multa, pelo descumprimento dos prazos limites estipulados nos itens anteriores, será aplicada multa de 0,1% sobre o valor total do contrato, além da multa estipulada no item originalmente descumprido.
> 13.2.5 Suspensão temporária de participação em licitação e impedimento de contratar com a Administração, por até 02 (dois) anos;
> 13.2.6. Declaração de inidoneidade para licitar ou contratar com a Administração Pública enquanto perdurarem os motivos determinantes da punição ou até que seja promovida a reabilitação perante a própria autoridade que aplicou a penalidade, que será concedida sempre que a licitante vencedora ressarcir a Administração pelos prejuízos resultantes e após decorrido o prazo da sanção aplicada com base no item anterior.
> 13.3 - As sanções previstas nos subitens, 13.1 e 13.2.1 poderão ser aplicadas concomitantemente com as dos subitens 13.2.2 e 13.2.3, facultada a defesa prévia do interessado no prazo de 5 (cinco) dias úteis, contados a partir de sua ciência;
> 13.4 - As sanções serão obrigatoriamente registradas no Sistema de Cadastramento Unificado de Fornecedores – SICAF;
> 13.5 - Em qualquer hipótese de aplicação de sanções serão assegurados à Adjudicatária o contraditório e a ampla defesa.
> 13.6 Comprovado o impedimento ou reconhecida força maior, devidamente justificado e aceito pelo CNMP, a CONTRATADA ficará isenta das penalidades mencionadas.
> 13.7 A multa, aplicada após regular processo administrativo, será cobrada administrativamente deduzindo-se do valor da Fatura devida à Contratada e, não sendo suficiente, o valor poderá ser inscrito como Dívida Ativa e cobrado judicialmente.
> 13.8 Previamente à aplicação das multas previstas nos subitens acima ou de qualquer outra sanção, poderá a Contratada apresentar defesa, no prazo de 5 (cinco) dias úteis, contado a partir da data em que for notificada a respeito.
> 13.9 Da aplicação das sanções previstas no art. 87 da Lei nº 8.666/93 e demais penalidades previstas no presente Contrato, caberá recurso, representação ou pedido de reconsideração, nos termos do art. 109 da supramencionada Lei, sendo assegurados o direito ao contraditório e à ampla defesa.
> 13.10 O setor responsável pelo recebimento do objeto deste Contrato deverá informar à Coordenadoria de Administração a ocorrência de fatos que possam motivar a aplicação das sanções previstas nesta Cláusula, sob pena de responsabilidade.
> 13.11 Os atos administrativos de aplicação das sanções previstas na Cláusula Décima Terceira, bem como a rescisão contratual, serão publicados resumidamente no Diário Oficial da União. 13.12 Os recursos relativos à aplicação das penalidades previstas nesta Cláusula Décima Terceira e no art. 87 da Lei nº 8.666/93, exceto para aquela definida no inciso IV da referida Lei, poderão ocorrer no prazo máximo de 5 (cinco) dias úteis a contar da intimação do ato ou da lavratura da ata.
> 13.13 No caso de declaração de inidoneidade, prevista no inciso IV do art. 87 da Lei nº 8.666/93, caberá pedido de reconsideração no prazo de 10 (dez) dias úteis a contar da intimação do ato .
> 13.14 Os autos do processo permanecerão com vista franqueada aos interessados, no Edifício-Sede II deste Conselho, sito: SAS, Quadra 03, Bloco J – Sala 110, Asa Sul, Brasília-DF.
> 
> CLÁUSULA DÉCIMA QUARTA – DA RESCISÃO
> 14.1 A inexecução total ou parcial do contrato poderá ensejar a sua rescisão, conforme disposto nos artigos 77 a 80 da Lei nº 8.666/1993.
> 14.2 Os casos de rescisão contratual serão formalmente motivados nos autos do procedimento, assegurados o contraditório e a ampla defesa.
> 14.3 A rescisão do contrato poderá ser:
> 14.3.1 determinada por ato unilateral e escrito da Contratante nos casos enumerados nos incisos I a XII e XVII do artigo 78 da Lei n.º 8.666/93, mediante notificação através de ofício entregue diretamente ou por via postal, com prova de recebimento, sem prejuízo das penalidades previstas neste contrato;
> 14.3.2 amigável, por acordo entre as partes, mediante a assinatura de termo aditivo ao contrato, desde que haja conveniência para a Contratante e
> 14.3.3 judicial, nos termos da legislação.
> 14.4 A rescisão unilateral ou amigável deverá ser precedida de autorização escrita e fundamentada da autoridade competente.
> 14.5 De conformidade com o § 2º do artigo 79, da Lei nº 8.666/93, quando a rescisão ocorrer com base nos incisos XII a XVII do artigo 78 da mesma lei, sem que haja culpa do contratado, será este ressarcido dos prejuízos regularmente comprovados que houver sofrido, tendo ainda direito aos pagamentos devidos pela execução do contrato até a data da rescisão;
> 14.6 A rescisão poderá acarretar na retenção dos créditos decorrentes do contrato, até o limite dos prejuízos causados à CONTRATANTE.
> 
> CLÁUSULA DÉCIMA QUINTA – DA VALIDADE
> 15.1. Este Contrato somente terá validade depois de aprovado pelo Secretário-Geral do Conselho Nacional do Ministério Público, no uso da competência que lhe foi atribuída pelo inciso VII, do artigo 2º, da Portaria n.º 94 de 14/12/2010, do Exmo. Presidente do CNMP, e publicado seu extrato no Diário Oficial da União, conforme dispõe o artigo 20 do Decreto n.º 3.555, de 8/8/2000.
> 
> CLÁUSULA DÉCIMA SEXTA - DA PUBLICIDADE
> 16.1 Incumbirá ao Contratante, por sua conta e no prazo estipulado no artigo 20 do Decreto n.º 3.555, de 8/8/2000, a publicação de extrato deste Contrato e de seus eventuais Termos Aditivos no Diário Oficial da União.
> 
> CLÁUSULA DÉCIMA SÉTIMA – DO FORO
> 17.1 O Foro da Justiça Federal de Brasília/DF é o competente para dirimir quaisquer dúvidas que vierem a surgir no cumprimento das obrigações aqui estabelecidas.
> 
> E, por estarem de pleno acordo, depois de lido e achado conforme, foi o presente Contrato lavrado em 03 (três) vias de igual teor e forma, assinado pelas partes juntamente com as testemunhas abaixo.
> 
> Brasília - DF, de de 2011.
> CONTRATANTE CONTRATADA
> TESTEMUNHAS:
> CPF: CPF:
> APROVO
> ____________________________
> ```
> 
> O contrato fornecido é um contrato para aquisição de material de informática. O texto do contrato foi extraído e formatado para facilitar a análise.
> 


-----------------------------

--- 🔎 Jargões Identificados ---


> Com base no texto do contrato fornecido, listo os seguintes jargões jurídicos e termos técnicos que provavelmente seriam desconhecidos por uma pessoa leiga:
> 
> 1.  **CNPJ**: Cadastro Nacional de Pessoa Jurídica.
> 2.  **CPF**: Cadastro de Pessoa Física.
> 3.  **RG**: Registro Geral (Documento de Identidade).
> 4.  **Pregão Eletrônico**: Modalidade de licitação realizada por meio eletrônico.
> 5.  **Lei nº 8.666/1993 (Lei de Licitações)**: Lei que estabelece normas gerais sobre licitações e contratos administrativos.
> 6.  **Lei nº 10.520/2002**: Lei que institui o pregão como modalidade de licitação.
> 7.  **Decreto nº 3.555/2000**: Regulamenta a Lei do Pregão.
> 8.  **Decreto nº 5.450/2005**: Dispõe sobre o pregão eletrônico.
> 9.  **Decreto nº 5.504/2005**: Estabelece a obrigatoriedade de utilização do pregão eletrônico.
> 10. **Lei Complementar nº 123/2006**: Trata do Estatuto Nacional da Microempresa e da Empresa de Pequeno Porte.
> 11. **Termo de Referência**: Documento que detalha o objeto da licitação, especificações, prazos, etc.
> 12. **Regime de Execução Indireta**: Forma de execução do contrato onde a contratada subcontrata terceiros.
> 13. **Empreitada por Preço Unitário**: Modalidade de contratação onde o preço é fixado por unidade de produto ou serviço.
> 14. **Habilitação e Qualificação**: Requisitos que a empresa deve cumprir para participar da licitação e celebrar o contrato.
> 15. **Atestar a execução do objeto**: Confirmar que o objeto do contrato foi executado conforme o combinado.
> 16. **Almoxarifado**: Local de armazenamento de materiais.
> 17. **Ônus**: Encargo, obrigação financeira.
> 18. **Termo de Recebimento Provisório**: Documento que formaliza o recebimento inicial dos bens ou serviços.
> 19. **Termo de Recebimento Definitivo**: Documento que formaliza o recebimento final dos bens ou serviços, após verificação de conformidade.
> 20. **Vigência**: Período de validade do contrato.
> 21. **Dotação Orçamentária**: Recursos financeiros previstos no orçamento para a execução do contrato.
> 22. **Natureza da Despesa**: Classificação orçamentária da despesa.
> 23. **Nota de Empenho**: Documento que garante a reserva de recursos para o pagamento do contrato.
> 24. **Termos Aditivos**: Instrumentos utilizados para alterar o contrato original.
> 25. **Ampla Defesa e Contraditório**: Princípios constitucionais que garantem o direito de se defender em um processo administrativo.
> 26. **SICAF**: Sistema de Cadastramento Unificado de Fornecedores.
> 27. **Inexecução total ou parcial**: Não cumprimento total ou parcial das obrigações contratuais.
> 28. **Multa Moratória**: Penalidade aplicada por atraso no cumprimento das obrigações.
> 29. **Cominações Legais**: Sanções previstas em lei para o descumprimento de obrigações.
> 30. **Inidoneidade**: Qualificação negativa para licitar ou contratar com a administração pública.
> 31. **Adjudicatária**: Empresa vencedora da licitação.
> 32. **Força Maior**: Evento imprevisível e inevitável que impede o cumprimento das obrigações contratuais.
> 33. **Dívida Ativa**: Créditos da administração pública que não foram pagos.
> 34. **Intimação**: Ato de comunicar oficialmente uma decisão ou fato a alguém.
> 35. **Diário Oficial da União**: Veículo oficial de publicação dos atos do governo federal.
> 36. **Rescisão Contratual**: Cancelamento do contrato.
> 37. **Ato Unilateral**: Decisão tomada por uma das partes sem a necessidade de acordo com a outra.
> 38. **Termo Aditivo**: Alteração contratual realizada por meio de um aditivo ao contrato original.
> 39. **Foro**: Local onde as ações judiciais devem ser propostas.
> 40. **Dirimir**: Solucionar, resolver.
> 41. **Instrução Normativa SRF nº 480/2004 e 539/2005**: Normas da Secretaria da Receita Federal sobre retenção de impostos.
> 42. **Lei Complementar nº 116/2003**: Dispõe sobre o Imposto Sobre Serviços (ISS).
> 43. **Sistema Integrado de Pagamento de Impostos e Contribuições da Microempresas e Empresas de Pequeno Porte – “SIMPLES” (Lei nº 9.317/96)**: Regime tributário simplificado para micro e pequenas empresas.
> 44. **Regularidade Fiscal**: Situação em que a empresa está em dia com suas obrigações fiscais.
> 45. **Inadimplência Contratual**: Não cumprimento das obrigações estabelecidas no contrato.
> 46. **Liquidação de Obrigação Financeira**: Quitação de uma dívida ou obrigação financeira.
> 47. **Encargos Moratórios**: Juros e outras despesas decorrentes do atraso no pagamento.
> 48. **Art. 38 do Decreto nº 93.872/86**: Dispositivo legal que trata de pagamentos antecipados.
> 49. **Art. 65 da Lei nº 8.666/93**: Artigo da Lei de Licitações que trata das alterações contratuais.
> 50. **Art. 28 do Decreto nº 5.450/2005**: Dispositivo legal que trata das penalidades no pregão eletrônico.
> 51. **Art. 78 da Lei nº 8.666/93**: Artigo da Lei de Licitações que trata dos motivos para rescisão contratual.
> 52. **Art. 87 da Lei nº 8.666/93**: Artigo da Lei de Licitações que trata das sanções administrativas.
> 53. **Art. 109 da Lei nº 8.666/93**: Artigo da Lei de Licitações que trata dos recursos administrativos.
> 54. **Artigos 77 a 80 da Lei nº 8.666/1993**: Artigos da Lei de Licitações que tratam da rescisão contratual.
> 55. **Artigo 20 do Decreto n.º 3.555, de 8/8/2000**: Dispositivo legal que trata da publicação do extrato do contrato no DOU.
> 
> 


-------------------------------

--- 💡 Explicações dos Termos ---


> Com prazer! Abaixo, as explicações dos termos listados, em linguagem acessível:
> 
> 1.  **CNPJ**: É como se fosse o "RG" de uma empresa. É um número que identifica a empresa na Receita Federal e é usado para tudo, desde abrir conta no banco até emitir notas fiscais.
> 2.  **CPF**: É o Cadastro de Pessoa Física, o número de identificação de cada cidadão brasileiro na Receita Federal. Assim como o CNPJ para empresas, o CPF é essencial para diversas atividades, como declarar imposto de renda e fazer compras parceladas.
> 3.  **RG**: É o Registro Geral, o documento de identidade que todos os brasileiros recebem. Ele contém informações como nome, data de nascimento, filiação e foto, e é usado para identificação em diversas situações.
> 4.  **Pregão Eletrônico**: Imagine uma compra governamental feita pela internet. Várias empresas dão seus lances online, e quem oferece o menor preço ganha. É uma forma de licitação mais rápida e transparente.
> 5.  **Lei nº 8.666/1993 (Lei de Licitações)**: É como um manual de regras para o governo fazer compras e contratar serviços. Ela garante que tudo seja feito de forma justa e transparente, evitando corrupção.
> 6.  **Lei nº 10.520/2002**: Essa lei criou o "pregão", uma forma mais ágil de licitação, ideal para compras e serviços comuns. Pense nela como uma versão mais moderna e eficiente da Lei de Licitações tradicional.
> 7.  **Decreto nº 3.555/2000**: Esse decreto detalha como funciona o pregão, desde a publicação do edital até a escolha do vencedor. Ele ajuda a garantir que o pregão seja feito de forma correta e transparente.
> 8.  **Decreto nº 5.450/2005**: Este decreto focou em como fazer o pregão pela internet, o chamado "pregão eletrônico". Ele trouxe mais agilidade e economia para as compras do governo.
> 9.  **Decreto nº 5.504/2005**: Determinou que, sempre que possível, o governo deve usar o pregão eletrônico em vez de outras formas de licitação. Isso ajudou a popularizar o pregão eletrônico e a economizar dinheiro público.
> 10. **Lei Complementar nº 123/2006**: É uma lei que dá vantagens para micro e pequenas empresas em licitações. Por exemplo, elas podem ter preferência em caso de empate com empresas maiores.
> 11. **Termo de Referência**: É um documento que explica tim-tim por tim-tim o que o governo quer comprar ou contratar. Ele detalha as especificações, prazos, e tudo mais que a empresa precisa saber para fazer sua proposta.
> 12. **Regime de Execução Indireta**: É quando a empresa contratada pelo governo "passa o serviço" para outras empresas menores. Ela continua responsável, mas quem faz o trabalho na prática são outras empresas.
> 13. **Empreitada por Preço Unitário**: Imagine que o governo quer construir um muro. Em vez de pagar um preço fixo pelo muro todo, ele paga por metro construído. Assim, o preço final depende do tamanho do muro.
> 14. **Habilitação e Qualificação**: Antes de uma empresa participar de uma licitação, ela precisa provar que tem condições de fazer o serviço. É como se fosse uma "carteira de motorista" para obras e serviços públicos.
> 15. **Atestar a execução do objeto**: É quando o governo confirma que a empresa fez o trabalho direitinho, conforme combinado no contrato. É como um "ok" final antes de pagar a empresa.
> 16. **Almoxarifado**: É o depósito onde são guardados materiais de construção, equipamentos, e tudo mais que é usado em uma obra ou serviço.
> 17. **Ônus**: É qualquer tipo de encargo ou obrigação financeira que uma empresa tem, como impostos, taxas, ou dívidas.
> 18. **Termo de Recebimento Provisório**: É um documento que o governo assina quando recebe os bens ou serviços pela primeira vez. Ele indica que está tudo ok, mas ainda precisa de uma verificação final.
> 19. **Termo de Recebimento Definitivo**: É o documento final que o governo assina após verificar que os bens ou serviços estão perfeitos. Ele libera o pagamento para a empresa.
> 20. **Vigência**: É o período em que o contrato é válido. Ele começa em uma data e termina em outra, e durante esse tempo as regras do contrato devem ser seguidas.
> 21. **Dotação Orçamentária**: É o dinheiro que o governo separou no orçamento para pagar por um contrato específico. Sem dotação, não tem como pagar a empresa!
> 22. **Natureza da Despesa**: É um código que indica o tipo de gasto que o governo está fazendo. Por exemplo, se está pagando por materiais de construção ou por serviços de limpeza.
> 23. **Nota de Empenho**: É um documento que garante que o dinheiro para pagar o contrato está reservado. É como se o governo dissesse: "Pode fazer o serviço, o dinheiro está garantido!".
> 24. **Termos Aditivos**: São documentos que alteram o contrato original. Eles podem mudar prazos, preços, ou qualquer outra coisa que precise ser ajustada.
> 25. **Ampla Defesa e Contraditório**: São direitos que garantem que uma pessoa pode se defender de uma acusação. Em um processo administrativo, por exemplo, a empresa tem o direito de apresentar sua versão dos fatos e contestar as acusações.
> 26. **SICAF**: É um sistema do governo onde as empresas se cadastram para poder participar de licitações. É como um "cadastro único" de fornecedores do governo.
> 27. **Inexecução total ou parcial**: É quando a empresa não cumpre o contrato, seja em parte ou totalmente. Isso pode gerar multas e outras penalidades.
> 28. **Multa Moratória**: É uma multa que a empresa paga por atrasar a entrega de um produto ou serviço. É como uma "taxa de atraso" para contratos.
> 29. **Cominações Legais**: São as punições que a lei prevê para quem não cumpre um contrato. Podem ser multas, suspensão, ou até mesmo proibição de contratar com o governo.
> 30. **Inidoneidade**: É quando uma empresa é considerada "não confiável" para contratar com o governo. Ela fica proibida de participar de licitações e fechar contratos.
> 31. **Adjudicatária**: É a empresa que venceu a licitação e tem o direito de assinar o contrato com o governo.
> 32. **Força Maior**: É um evento imprevisível e inevitável que impede a empresa de cumprir o contrato. Por exemplo, uma enchente que destrói a obra. Nesses casos, a empresa pode ser isenta de penalidades.
> 33. **Dívida Ativa**: É o dinheiro que uma pessoa ou empresa deve ao governo e não pagou. O governo pode cobrar essa dívida na Justiça.
> 34. **Intimação**: É um aviso oficial que o governo envia para uma pessoa ou empresa. Ele pode ser para pagar uma dívida, comparecer a uma audiência, ou qualquer outra coisa.
> 35. **Diário Oficial da União**: É o jornal oficial do governo federal. Nele são publicados leis, decretos, portarias, e outros atos do governo.
> 36. **Rescisão Contratual**: É o cancelamento do contrato. Ele pode ser feito por acordo entre as partes ou por decisão unilateral do governo.
> 37. **Ato Unilateral**: É uma decisão que uma das partes toma sem precisar da concordância da outra. Por exemplo, o governo pode rescindir um contrato unilateralmente se a empresa não cumprir suas obrigações.
> 38. **Termo Aditivo**: É um documento que modifica um contrato já existente. Ele pode alterar prazos, preços, ou qualquer outra condição do contrato.
> 39. **Foro**: É o local onde um processo judicial deve ser iniciado. Em um contrato, as partes podem escolher o foro onde resolverão eventuais disputas.
> 40. **Dirimir**: Significa resolver um conflito ou dúvida. Em um contrato, as partes podem definir como dirimir eventuais dúvidas na interpretação do contrato.
> 41. **Instrução Normativa SRF nº 480/2004 e 539/2005**: São normas da Receita Federal que explicam como as empresas devem reter impostos nos pagamentos que fazem a outras empresas.
> 42. **Lei Complementar nº 116/2003**: É a lei que trata do Imposto Sobre Serviços (ISS), um imposto que as empresas pagam quando prestam serviços.
> 43. **Sistema Integrado de Pagamento de Impostos e Contribuições da Microempresas e Empresas de Pequeno Porte – “SIMPLES” (Lei nº 9.317/96)**: É um regime tributário simplificado para micro e pequenas empresas. Ele unifica o pagamento de vários impostos em uma única guia, facilitando a vida dos empresários.
> 44. **Regularidade Fiscal**: É quando a empresa está em dia com o pagamento de seus impostos e contribuições. Para contratar com o governo, a empresa precisa estar em situação de regularidade fiscal.
> 45. **Inadimplência Contratual**: É quando uma das partes não cumpre as obrigações estabelecidas no contrato. Isso pode gerar multas, rescisão do contrato, e outras penalidades.
> 46. **Liquidação de Obrigação Financeira**: É o pagamento de uma dívida ou obrigação financeira. Quando uma empresa paga uma conta, ela está liquidando uma obrigação financeira.
> 47. **Encargos Moratórios**: São os juros e multas que uma pessoa ou empresa paga quando atrasa o pagamento de uma dívida.
> 48. **Art. 38 do Decreto nº 93.872/86**: É um artigo de um decreto que trata dos casos em que o governo pode fazer pagamentos antecipados em um contrato.
> 49. **Art. 65 da Lei nº 8.666/93**: É um artigo da Lei de Licitações que trata das alterações que podem ser feitas em um contrato.
> 50. **Art. 28 do Decreto nº 5.450/2005**: É um artigo de um decreto que trata das punições que podem ser aplicadas em um pregão eletrônico.
> 51. **Art. 78 da Lei nº 8.666/93**: É um artigo da Lei de Licitações que lista os motivos que podem levar à rescisão de um contrato.
> 52. **Art. 87 da Lei nº 8.666/93**: É um artigo da Lei de Licitações que trata das sanções administrativas que podem ser aplicadas a empresas que não cumprem contratos com o governo.
> 53. **Art. 109 da Lei nº 8.666/93**: É um artigo da Lei de Licitações que trata dos recursos que as empresas podem apresentar quando não concordam com uma decisão do governo em uma licitação.
> 54. **Artigos 77 a 80 da Lei nº 8.666/1993**: São artigos da Lei de Licitações que tratam da rescisão de contratos administrativos. Eles definem os motivos para a rescisão, os direitos das partes, e os procedimentos a serem seguidos.
> 55. **Artigo 20 do Decreto n.º 3.555, de 8/8/2000**: É um artigo de um decreto que exige que o governo publique um resumo do contrato no Diário Oficial da União. Isso garante a transparência e permite que a sociedade acompanhe os gastos públicos.
> 
> Espero que essas explicações ajudem a entender melhor o contrato! Se tiver mais alguma dúvida, é só perguntar.
> 


---------------------------------

--- 🔑 Resumo e Pontos Chave do Contrato ---


> ## Resumo do Contrato de Aquisição de Material de Informática
> 
> Este contrato tem como objetivo a aquisição de material de informática pelo Conselho Nacional do Ministério Público (CNMP) junto à empresa contratada. O contrato detalha as obrigações de ambas as partes, os prazos e condições de entrega, o valor total do contrato e as condições de pagamento. Além disso, estabelece as penalidades em caso de descumprimento e as condições para rescisão do contrato.
> 
> ## Pontos Chave do Contrato
> 
> *   **Objeto do Contrato:** Aquisição de material de informática, incluindo garantia e assistência técnica, conforme especificações do edital do Pregão Eletrônico CNMP nº 49/2011.
> *   **Obrigações da Contratada:**
>     *   Entregar o material conforme especificado e dentro do prazo.
>     *   Prestar garantia e assistência técnica.
>     *   Pagar todos os impostos e taxas incidentes.
>     *   Apresentar notas fiscais corretas.
>     *   Manter as condições de habilitação e qualificação.
> *   **Obrigações do Contratante (CNMP):**
>     *   Permitir o acesso da contratada ao local de entrega.
>     *   Atestar a execução do contrato.
>     *   Efetuar o pagamento conforme o contrato.
>     *   Fiscalizar a execução do contrato.
> *   **Prazos e Condições:**
>     *   Entrega: 30 dias corridos após a confirmação do pedido.
>     *   Recebimento provisório: até 5 dias úteis após a entrega.
>     *   Recebimento definitivo: até 8 dias úteis após o recebimento provisório.
> *   **Valor e Pagamento:**
>     *   Valor total: R$ XXX,00.
>     *   Pagamento: em até 10 dias úteis após o recebimento definitivo, mediante apresentação da nota fiscal.
> *   **Rescisão:** O contrato pode ser rescindido em caso de descumprimento das cláusulas ou por outros motivos previstos na Lei nº 8.666/1993.
> *   **Penalidades:**
>     *   Multa por atraso na entrega: 0,7% por dia de atraso.
>     *   Multa por atraso no atendimento técnico: 0,3% por dia de atraso.
>     *   Suspensão temporária de participação em licitação e impedimento de contratar com a Administração.
>     *   Declaração de inidoneidade para licitar ou contratar com a Administração Pública.
> 
> 


-------------------------------------------

✅ Análise do Contrato Concluída!
